In [23]:
import numpy as np

file_path = "1a91.pdb"

coordinates_protein = []
coordinates_membrane = []
with open(file_path,"r") as f:

    line = f.readline()

    while line:

        line = line.split()

        if line[0] == "ATOM" and line[2] == "CA":

            x = float(line[6])
            y = float(line[7])
            z = float(line[8])

            coordinates_protein.append([x,y,z])

        elif line[0] == "HETATM" and "DUM" in line:

            x = float(line[5])
            y = float(line[6])
            z = float(line[7])

            coordinates_membrane.append([x,y,z])

        line = f.readline()

coordinates_protein = np.array(coordinates_protein)
coordinates_membrane = np.array(coordinates_membrane)

# Is the protein transmembrane?

# The protein is transmembrane if it has at least one residue with a z coordinate
# greater than the z coordinate of the membrane

z_protein = coordinates_protein[:,2]
z_membrane = coordinates_membrane[:,2]


if np.any(z_protein > np.max(z_membrane)) and np.any(z_protein < np.min(z_membrane)):
    print("The protein is transmembrane")
    

The protein is transmembrane
